In [ ]:
import subprocess

result = subprocess.run(
    ['echo', '-e', 'Hello from child']
)

result.check_returncode()
print(result.stdout)

In [ ]:
result.check_returncode()

In [ ]:
print(result.stdout)

In [ ]:
proc = subprocess.Popen(['sleep', '10'])
while proc.poll() is None:
    pass 


In [ ]:
import time 

start = time.time()
sleep_procs = []

for _ in range(10):
    proc = subprocess.Popen(['sleep', '1'])
    sleep_procs.append(proc)

    

In [ ]:
for proc in sleep_procs:
    proc.communicate()
end = time.time()

print(end - start)

In [ ]:
import os 
def run_encryt(data):
    env = os.environ.copy()
    env['password'] = 'zf7ShyBhZOraQDdE/FiZpm/m/8f9X+M1'
    proc = subprocess.Popen(
        ['openssl', 'enc', '-des3', '-pass', 'env:password'],
        env=env,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
    )
    proc.stdin.write(data)
    return proc 

In [ ]:
procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_encryt(data)
    procs.append(proc)


In [ ]:
for proc in procs:
    out, _ = proc.communicate()
    print(out[:])
# print(help(proc.communicate))

In [ ]:
def run_hash(input_stdin):
    return subprocess.Popen(
        ['openssl', 'dgst', '-sha256', '-binary'],
        stdin=input_stdin,
        stdout=subprocess.PIPE
    )

In [ ]:
encr_procs = []
hash_procs = []

for _ in range(3):
    data = os.urandom(100)
    
    encrypt_proc = run_encryt(data)
    encr_procs.append(encrypt_proc)
    
    hash_proc = run_hash(encrypt_proc.stdout)
    hash_procs.append(hash_proc)
    
    encrypt_proc.stdout.close()
    encrypt_proc.stdout = None
    
for proc in encr_procs:
    out, _ = proc.communicate()
    # print(out)
    assert proc.returncode == 0
    
for proc in hash_procs:
    out, _ = proc.communicate()
    print(out)
    assert proc.returncode == 0

In [ ]:
proc =subprocess.Popen(['sleep', '10'])

try:
    proc.communicate(timeout=0.1)
except subprocess.TimeoutExpired:
    proc.terminate()
    proc.wait()

print('Exit status', proc.poll())


In [ ]:
subprocess.run(
    ['echo', '-n', 'Hello from the child'],
    capture_output = True, 
    encoding='utf-8'
).stdout


In [ ]:
def factorize(numbers):
    for i in range(1, numbers + 1):
        if numbers % i == 0:
            yield i


In [ ]:
numbers = [2139079, 1214759, 1516637, 1852285]
start = time.time()

for number in numbers:
    list(factorize(number))
    
end = time.time()

delta = end - start
print(f"Took {delta:.3f} seconds")

In [ ]:
from threading import Thread

class FactorizeThread(Thread):
    def __init__(self, number):
        super().__init__()
        self.number = number 

    def run(self):
        self.factors = list(factorize(self.number))
        

In [ ]:
start = time.time()

threads = []
for number in numbers:
    thread = FactorizeThread(number)
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()
    
end = time.time()
delta = end - start
print(f"Took {delta:.3f} seconds")

In [ ]:
import socket, select 

def slow_systemcall():
    print("sleeping")
    select.select([socket.socket()], [],[], 1)

start = time.time()

for _ in range(5):
    slow_systemcall()

end = time.time()
print(f"Took {end - start:.3f} seconds")

# this method will keep GIL to block the main execution thread

In [ ]:
start = time.time()
threads = []
for _ in range(5):
    thread = Thread(target=slow_systemcall)
    thread.start()
    threads.append(thread)


def compute_helicopter_location(index):
    return index * 0.5

for i in range(5):
    compute_helicopter_location(i)

for thread in threads:
    thread.join()

end = time.time()
print(f"Took {end - start:.3f} seconds")

##### all systemcalls will run in parallel with main thread
    Python threads can’t run in parallel on multiple CPU cores because of the global interpreter lock (GIL)

In [ ]:
class Counter:
    def __init__(self):
        self.count = 0

    def increments(self, offset):
        self.count += offset

In [ ]:
def worker(sensor_index, how_many, counter):
    for _ in range(how_many):
        counter.increments(1)

In [ ]:
from threading import Thread  

how_many = 10 ** 5 
counter = Counter()

threads = []
for i in range(5):
    thread = Thread(target=worker,
                    args=(i, how_many, counter))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()
    
expected = 10 ** 5 * 5
found = counter.count 
print(f"Counter should be {expected}, found {found}")

In [ ]:
from threading import Thread
import socket 

def run_server(host="localhost", port=33333):
    sock = socket.socket()
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock.bind((host, port))
    sock.listen()
    
    while True:
        client_sock, addr = sock.accept()
        print("Connection from ", addr)
        Thread(target=handle_client, args=(client_sock,)).start()


def handle_client(sock):
    print(sock)
    while True:
        received_data = sock.recv(4096)
        if not received_data:
            break
        sock.sendall(received_data)
        
    print("Client disconnected:", sock.getpeername())
    sock.close()
    
def compute():
    n = 0
    while True:
        n+=1
        n-=1

Thread(target=compute).start()
run_server()

In [ ]:
# python is single threaded so performance will down to 300 times 

from multiprocessing import Process

# Process(target=compute).start()
run_server(port=8081)

In [ ]:
# sys.setswitchinterval(1)
from threading import Thread 

In [ ]:
Thread(target=lambda : print("Hello")).start().join()
while True:
    print("Hello")